In [421]:
#Importing Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

### Create an empty dataframe to store foreign data

In [422]:
fxdata = pd.DataFrame()
pd.options.mode.chained_assignment = None #Prevent error Warning of a copy in memory

### Web scrapper to pull fx data from Abokifx.com

In [423]:
count = 0 
while count < 10    :
    res = requests.get('https://abokifx.com/ratetypes/?rates=cbn')#Fx rate is at CBN rate
    soup = BeautifulSoup(res.content,'lxml') #Use Beautiful soup to prettify our data
    table = soup.find_all('table')[0] #Find all the html tag named 'table'
    html_data = pd.read_html(str(table), index_col = False)[0] #Read the content of found html tables 
    fxdata = fxdata.append(html_data) # Append to the empty Dataframe above
    print("page "+str(count)+" Done.") #Keep track of each web page read
    count = count + 1

page 0 Done.
page 1 Done.
page 2 Done.
page 3 Done.
page 4 Done.
page 5 Done.
page 6 Done.
page 7 Done.
page 8 Done.
page 9 Done.


### Set new columns to replace the default column

In [424]:
fxdata.columns = ['Date', 'USD', 'GBP', 'EUR']

### Drop the first row not consistent with our data

In [425]:
fxdata.drop(0, axis = 0, inplace = True)

In [426]:
#fxdata.to_csv('Foreign_exchange_data.csv') # Option to Export the pulled fx data and store a CSV file

### Let's drop Euro. We're working with Dollars and Pounds alone

In [427]:
fxdata.drop('EUR', axis = 1, inplace = True)

### Convert our USD and GBP columns to Float values

In [428]:
fxdata['USD'] = fxdata['USD'].astype(np.float16)
fxdata['GBP'] = fxdata['GBP'].astype(np.float16)

In [429]:
fxdata.head()

,Date,USD,GBP
1,14/11/2018,306.75,397.75
2,13/11/2018,306.75,396.50
3,12/11/2018,306.75,394.75
4,09/11/2018,306.75,399.50
5,08/11/2018,306.75,402.00


In [430]:
fxdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 350 entries, 1 to 35
Data columns (total 3 columns):
Date    350 non-null object
USD     350 non-null float16
GBP     350 non-null float16
dtypes: float16(2), object(1)
memory usage: 6.8+ KB


### Create a new column for computing the foreign exchage rate ratio

In [431]:
fxdata['USD_Ratio'] = fxdata['USD']/fxdata['GBP']
fxdata['BP_Ratio'] = fxdata['GBP']/fxdata['USD']

In [432]:
fxdata.head()

,Date,USD,GBP,USD_Ratio,BP_Ratio
1,14/11/2018,306.75,397.75,0.770996,1.296875
2,13/11/2018,306.75,396.50,0.773438,1.292969
3,12/11/2018,306.75,394.75,0.776855,1.287109
4,09/11/2018,306.75,399.50,0.768066,1.302734
5,08/11/2018,306.75,402.00,0.763184,1.310547


In [439]:
from sklearn import preprocessing, cross_validation, svm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

### Build the machine Learning model

In [440]:
x = fxdata.drop(['BP_Ratio', 'Date'], axis = 1)
y = fxdata['BP_Ratio']

In [441]:
x = preprocessing.scale(x)

### Splitting the data into a train and a test set

In [442]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state = 42)

### Using Linear Regression Algorithm

In [443]:
linear_model = LinearRegression()
linear_model.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

### Predictions

In [445]:
predictions = (linear_model.predict(x_test))

### Model evaluation

In [456]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, predictions))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, predictions))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

Mean Absolute Error: 0.0001981481895668981
Mean Squared Error: 5.8405681999784184e-08
Root Mean Squared Error: 0.0002416726753271544


### Using Support Vector Regression (SVR) Machine Learning Algorithm

In [448]:
from sklearn import svm

In [450]:
svr_model = svm.SVR() #Creating an instance of SVR model
svr_model.fit(x_train, y_train)   # Fitting the model on the data

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [451]:
pred = svr_model.predict(x_test)

### Model Evaluation

In [457]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, pred)))

Mean Absolute Error: 0.010169719827586207
Mean Squared Error: 0.00016050503171723466
Root Mean Squared Error: 0.012669058043802413


<h2>Summary of Model Performance</h2>
<p> From the perfomance of both models above, one can see that the Error in predicting the Pounds to Dollar ratio is relatively small</p>
<p> One can have some level of confidence on the outcome of the prediction of foreign exchange ratio.</p>
<p> The prediction values below is the result of carraying out a predictive analysis in the foreign exchange data to test the model in production, one can easily feed new set of foreign exchange data to the model and the model will return a new set of predicted values.</p>

### Predicted Values:

In [489]:
date = fxdata['Date'][:116]

In [496]:
predictions_df = pd.DataFrame(predictions,date)#Convert predicted values to a data frame
predictions_df.to_csv('Predicted Values.csv', header = ['Predicted BPRatio'])